# GPT Model

**Descrição:**
Notebook dedicado à implementação e exploração de um **GPT decoder-only** (`GPTModel`) em PyTorch: combina *token embeddings* e *positional embeddings*, empilha blocos Transformer com **atenção causal (masked self-attention)** e finaliza com uma projeção linear para produzir **logits sobre o vocabulário** em cada posição da sequência.

**Objetivo:**
Consolidar o entendimento prático de como um GPT gera texto via **previsão do próximo token**, validando o fluxo do `forward`, checando *shapes* dos tensores, conferindo reprodutibilidade (seed), avaliando recursos (GPU) e inspecionando métricas como **número de parâmetros treináveis**, além de executar testes simples de geração.

**Funcionamento:**
1. **Entrada**: `in_idx` com shape **(B, T)** (IDs inteiros de tokens).  
2. **Embeddings**: `tok_emb(in_idx)` → **(B, T, C)** e `pos_emb(0..T-1)` → **(T, C)**; soma e aplica *dropout*.  
3. **Transformer blocks (`trf_blocks`)**: processam o contexto com **atenção causal** (cada posição atende apenas `≤ t`), MLP/FFN, conexões residuais e normalização.  
4. **Saída**: `final_norm` e `out_head` projetam para **logits** com shape **(B, T, V)** (probabilidades só após `softmax`, tipicamente na geração).


![Bloco transformer](../../imagens/cap04/05_gpt_model.png)

*Baseado na figura do livro, Capítulo 4, página 118*

In [1]:
from typing import Any

import tiktoken
import torch
import torch.nn as nn

from build_llm.layer import LayerNorm
from build_llm.transformer import TransformerBlock

In [2]:
tokenizer = tiktoken.get_encoding("gpt2")

### Verifica a presença da GPU

In [3]:
print(torch.__version__)  # Versão do torch
print(torch.cuda.is_available())  # Verificação de GPU
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Usando:", device)

2.9.1+cpu
False
Usando: cpu


## Fixando o seed para reproducibilidade

In [4]:
def set_seed(seed: int = 42) -> None:
    """
    Fixa seeds para reprodutibilidade em Python, NumPy e PyTorch.
    """
    torch.manual_seed(seed)

    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

    # Garante determinismo (pode afetar performance)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


set_seed(123)

## Configuração do modelo

As mesmas utilizadas no notebook [01 - LLM architecture](./01%20-%20LLM%20architecture.ipynb)

In [5]:
GPT_CONFIG_124M: dict[str, Any] = {
    "vocab_size": 50257,  # Tamanho do vocabulário
    "context_length": 1024,  # Comprimento do contexto
    "emb_dim": 768,  # Dimensão do embedding
    "n_heads": 12,  # Número de cabeças de atenção
    "n_layers": 12,  # Número de camadas
    "drop_rate": 0.1,  # Taxa de dropout
    "qkv_bias": False,  # Viés em Query-Key-Value (QKV)
}

# Como o `GPTModel` funciona (passo a passo)

A classe `GPTModel` implementa um **Transformer decoder-only** (estilo GPT), treinado para **predizer o próximo token** a partir de um contexto à esquerda. Em termos práticos: para cada posição `t` da sequência, o modelo produz uma distribuição (logits) sobre o vocabulário para o “próximo token provável”. Esse é o mecanismo central do GPT descrito no capítulo de implementação do modelo (cap. 4).

## Intuição: o que entra e o que sai

- **Entrada:** `in_idx` com shape **(B, T)**  
  `B = batch_size`, `T = seq_len`  
  Cada elemento é um **ID inteiro** de token (ex.: 50256, 123, 99...).

- **Saída:** `logits` com shape **(B, T, V)**  
  `V = vocab_size`  
  Para cada posição da sequência, temos um vetor de logits com tamanho do vocabulário (um “score” para cada token possível).

## Por que embeddings?

Redes neurais não operam diretamente em IDs discretos. Então o GPT faz duas codificações iniciais e as soma:

### 1) `tok_emb`: Token Embedding
Transforma IDs em vetores densos:
- `tok_embeds = tok_emb(in_idx)`  
- shape: **(B, T, C)**, onde `C = emb_dim`

Cada token vira um vetor “semântico” treinável.

### 2) `pos_emb`: Positional Embedding
Sem posição, o modelo veria a sequência como um “saco de tokens”.
- cria `pos_ids = [0, 1, 2, ..., T-1]`
- `pos_embeds = pos_emb(pos_ids)`  
- shape: **(T, C)** (depois é broadcast para (B, T, C))

Cada posição ganha um vetor treinável que informa “onde” o token está.

### Soma + Dropout
- `x = tok_embeds + pos_embeds`  → **(B, T, C)**
- `x = drop_emb(x)`

A soma combina “o que é” (token) com “onde está” (posição). O dropout ajuda na regularização.

## O coração do modelo: `trf_blocks` (TransformerBlock empilhados)

```
x = self.trf_blocks(x)
```

Aqui acontece o processamento contextual: cada token pode “olhar” para outros tokens do contexto e ajustar sua representação.

Embora o seu `GPTModel` não mostre o conteúdo de `TransformerBlock`, um bloco GPT típico contém:

1. **Atenção causal (masked self-attention)**  
   - Cada posição `t` só pode atender **posições ≤ t**  
   - Isso impede “ver o futuro” e torna o modelo autoregressivo (essencial para previsão do próximo token). :contentReference[oaicite:1]{index=1}

2. **MLP / Feed-Forward**  
   - Uma rede totalmente conectada aplicada por token para refinar representações.

3. **Residuais + Normalização**  
   - Conexões de atalho (skip connections) preservam informação e estabilizam gradientes.
   - LayerNorm ajuda a estabilizar o treino.

Como você empilha `n_layers`, o modelo aprende dependências mais complexas e de longo alcance.

## Finalização: normalizar e projetar para o vocabulário

### `final_norm`

```
x = self.final_norm(x)
```
Uma última normalização antes de converter para logits. Em GPTs isso costuma melhorar a estabilidade e a qualidade das saídas.

### `out_head`: Linear para logits
```
logits = self.out_head(x)
```
- Entrada: **(B, T, C)**
- Saída: **(B, T, V)**

Essa projeção “transforma” cada vetor oculto em um vetor de scores para cada token do vocabulário.

> Importante: **logits não são probabilidades**. Para obter probabilidades você aplicaria `softmax` no eixo do vocabulário (geralmente só na hora de gerar texto).

## O que o modelo aprende no treino (visão operacional)

No treino de *next-token prediction*, você normalmente:
- fornece uma sequência de entrada
- pede ao modelo para prever o próximo token em cada posição

Exemplo conceitual:
- entrada:  `["Eu", "gosto", "de"]`
- alvo:     `["gosto", "de", "café"]`

O modelo produz logits para cada posição e a loss (ex.: cross-entropy) compara com os alvos deslocados.

Essa lógica é o fundamento do GPT: uma tarefa simples (prever próximo token) que, com dados e escala, leva a comportamento linguístico complexo. :contentReference[oaicite:2]{index=2}

## Checagem de shapes (resumo)

Considere `B=batch_size`, `T=seq_len`, `C=emb_dim`, `V=vocab_size`:

| Etapa | Tensor | Shape |
|------|--------|-------|
| entrada | `in_idx` | (B, T) |
| token emb | `tok_embeds` | (B, T, C) |
| pos emb | `pos_embeds` | (T, C) |
| soma | `x` | (B, T, C) |
| blocos | `x` | (B, T, C) |
| norm final | `x` | (B, T, C) |
| cabeça | `logits` | (B, T, V) |

## Por que o `forward` valida entrada?

O método valida:
- se `in_idx` é `torch.Tensor`
- se é 2D (B, T)

In [6]:
class GPTModel(nn.Module):
    """
    Implementação de um modelo GPT (decoder-only Transformer).

    Componentes:
    -----------
    - Embedding de tokens (tok_emb): converte IDs de tokens em vetores.
    - Embedding posicional (pos_emb): adiciona informação de posição na sequência.
    - Dropout (drop_emb): regularização nos embeddings somados.
    - Blocos Transformer (trf_blocks): pilha de TransformerBlock.
    - Normalização final (final_norm): LayerNorm antes da projeção final.
    - Cabeça de saída (out_head): projeção para logits no vocabulário.

    Parâmetros esperados em `cfg`:
    ------------------------------
    vocab_size : int
        Tamanho do vocabulário (número de tokens possíveis).
    emb_dim : int
        Dimensão dos embeddings / hidden size.
    context_length : int
        Comprimento máximo de contexto (seq_len máximo).
    drop_rate : float
        Taxa de dropout aplicada após a soma tok_emb + pos_emb.
    n_layers : int
        Número de blocos Transformer.

    Retorno do forward:
    -------------------
    torch.Tensor
        Logits com shape (batch_size, seq_len, vocab_size).
    """

    def __init__(self, cfg: dict[str, Any]) -> None:
        super().__init__()

        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])

        self.trf_blocks = nn.Sequential(
            *[TransformerBlock(cfg) for _ in range(cfg["n_layers"])]
        )

        self.final_norm = LayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(cfg["emb_dim"], cfg["vocab_size"], bias=False)

    def forward(self, in_idx: torch.Tensor) -> torch.Tensor:
        """
        Executa o forward pass do GPT.

        Parâmetros:
        ----------
        in_idx : torch.Tensor
            Tensor de IDs de tokens com shape (batch_size, seq_len) e dtype inteiro.

        Retorno:
        -------
        torch.Tensor
            Logits com shape (batch_size, seq_len, vocab_size).

        Exceções:
        --------
        Levanta ValueError se `in_idx` não for um tensor 2D.
        """
        if not isinstance(in_idx, torch.Tensor):
            raise TypeError("`in_idx` deve ser um torch.Tensor.")
        if in_idx.ndim != 2:
            raise ValueError("`in_idx` deve ter shape (batch_size, seq_len).")

        batch_size, seq_len = in_idx.shape  # batch_size não é usado diretamente aqui

        tok_embeds = self.tok_emb(in_idx)  # (B, T, C)

        pos_ids = torch.arange(seq_len, device=in_idx.device)
        pos_embeds = self.pos_emb(pos_ids)  # (T, C) -> broadcast p/ (B, T, C)

        x = tok_embeds + pos_embeds  # (B, T, C)
        x = self.drop_emb(x)

        x = self.trf_blocks(x)
        x = self.final_norm(x)

        logits = self.out_head(x)  # (B, T, vocab_size)
        return logits

## Teste do GPT Model

In [7]:
batch = []

txt1 = "O gato dorme no"

batch.append(torch.tensor(tokenizer.encode(txt1)))
batch = torch.stack(batch, dim=0)
print(batch)

tensor([[   46,   308,  5549, 18586,    68,   645]])


### Verificando os tensores

In [8]:
model = GPTModel(GPT_CONFIG_124M)

out = model(batch)
print("Input batch:\n", batch)
print("\nOutput shape:", out.shape)
print(out)

Input batch:
 tensor([[   46,   308,  5549, 18586,    68,   645]])

Output shape: torch.Size([1, 6, 50257])
tensor([[[-0.0545, -0.7861,  0.9106,  ...,  0.6177,  0.0392, -0.4710],
         [ 0.2858, -0.3389,  0.0768,  ..., -0.7126, -0.2354, -0.1229],
         [ 0.8791, -0.1072,  0.6407,  ..., -1.0492, -0.9322, -0.8358],
         [-0.5154,  0.2904,  0.4959,  ..., -0.8074, -0.2613, -0.7128],
         [ 0.5876,  0.0875,  0.1256,  ..., -1.2970, -0.0646, -0.7104],
         [ 0.1734,  0.3456,  0.5032,  ..., -0.7875,  1.3982, -1.1401]]],
       grad_fn=<UnsafeViewBackward0>)


### Número total de parâmetros treináveis

In [9]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Número total de parâmetros: {total_params:,}")

Número total de parâmetros: 163,009,536


In [10]:
print("Formato da camada de embeddings de tokens:", model.tok_emb.weight.shape)
print("Formato da camada de saída:", model.out_head.weight.shape)

# Calcula o tamanho total em bytes (assumindo float32, 4 bytes por parâmetro)
tamanho_total_bytes = total_params * 4

# Converte para megabytes
tamanho_total_mb = tamanho_total_bytes / (1024 * 1024)

print(f"Tamanho total do modelo: {tamanho_total_mb:.2f} MB")

Formato da camada de embeddings de tokens: torch.Size([50257, 768])
Formato da camada de saída: torch.Size([50257, 768])
Tamanho total do modelo: 621.83 MB


## Geração dos próximos tokens a partir de pequeno texto

In [11]:
def generate_text_simple(
    model: nn.Module,
    idx: torch.Tensor,
    max_new_tokens: int,
    context_size: int,
) -> torch.Tensor:
    """
    Gera texto de forma simples (greedy decoding), adicionando tokens um a um.

    A cada iteração:
    - Recorta o contexto para no máximo `context_size` tokens (janela deslizante).
    - Calcula os logits do modelo.
    - Usa apenas o último passo de tempo (último token) para decidir o próximo token.
    - Aplica softmax e seleciona o token de maior probabilidade (argmax).
    - Concatena o token escolhido à sequência.

    Parâmetros:
    ----------
    model : nn.Module
        Modelo que recebe um tensor de IDs (batch_size, seq_len) e retorna logits
        (batch_size, seq_len, vocab_size).
    idx : torch.Tensor
        Tensor de IDs de tokens do contexto atual, com shape (batch_size, n_tokens).
        Deve ser inteiro (tipicamente torch.long).
    max_new_tokens : int
        Número máximo de novos tokens a serem gerados.
    context_size : int
        Tamanho máximo do contexto suportado (janela usada como entrada do modelo).

    Retorno:
    -------
    torch.Tensor
        Tensor com a sequência estendida, shape (batch_size, n_tokens + max_new_tokens).

    Exceções:
    --------
    Levanta TypeError/ValueError para entradas inválidas (tipos, shapes, valores).
    """
    if not isinstance(model, nn.Module):
        raise TypeError("`model` deve ser uma instância de torch.nn.Module.")
    if not isinstance(idx, torch.Tensor):
        raise TypeError("`idx` deve ser um torch.Tensor.")
    if idx.ndim != 2:
        raise ValueError("`idx` deve ter shape (batch_size, n_tokens).")
    if not isinstance(max_new_tokens, int) or max_new_tokens < 0:
        raise ValueError("`max_new_tokens` deve ser um inteiro >= 0.")
    if not isinstance(context_size, int) or context_size <= 0:
        raise ValueError("`context_size` deve ser um inteiro > 0.")

    # Garante que idx esteja em um dtype inteiro comum para embeddings
    if idx.dtype not in (
        torch.int64,
        torch.int32,
        torch.int16,
        torch.int8,
        torch.uint8,
    ):
        raise TypeError("`idx` deve ser um tensor de inteiros (ex.: torch.long).")

    for _ in range(max_new_tokens):
        # Recorta o contexto para caber no tamanho máximo suportado
        idx_cond = idx[:, -context_size:]

        # Predição sem gradientes
        with torch.no_grad():
            logits = model(idx_cond)

        # Usa apenas o último passo de tempo: (B, T, V) -> (B, V)
        logits_last = logits[:, -1, :]

        # Probabilidades do próximo token: (B, V)
        probas = torch.softmax(logits_last, dim=-1)

        # Escolha greedy: token com maior probabilidade (B, 1)
        idx_next = torch.argmax(probas, dim=-1, keepdim=True)

        # Concatena o próximo token à sequência: (B, T) -> (B, T+1)
        idx = torch.cat((idx, idx_next), dim=1)

    return idx

In [12]:
start_context = "O gato sobe no"

encoded = tokenizer.encode(start_context)
print("encoded:", encoded)

encoded_tensor = torch.tensor(encoded).unsqueeze(0)
print("encoded_tensor.shape:", encoded_tensor.shape)

encoded: [46, 308, 5549, 523, 1350, 645]
encoded_tensor.shape: torch.Size([1, 6])


### Verificando os tensores de saída

In [16]:
model.eval()  # disable dropout

out = generate_text_simple(
    model=model,
    idx=encoded_tensor,
    max_new_tokens=6,  # 6 tokens além dos que já foram enviados
    context_size=GPT_CONFIG_124M["context_length"],
)

print("Output:", out)
print("Output length:", len(out[0]))

Output: tensor([[   46,   308,  5549,   523,  1350,   645, 34381, 37516, 12434, 42030,
         37383,   678]])
Output length: 12


### O texto saiu aleatório pois ainda não houve treinamento

In [17]:
decoded_text = tokenizer.decode(out.squeeze(0).tolist())
print(decoded_text)

O gato sobe no Dice eighty Driverfoundland Garner 19
